# OpenVLA evaluation on LIBERO benchmark

Section 1: Initialize OpenVLA with Huggingface transformer

In [ ]:
%env TRANSFORMERS_CACHE=/data2/zhaoyu/huggingface_cache
# os.environ['TRANSFORMERS_CACHE'] = '/data2/zhaoyu/huggingface_cache'
from transformers import AutoModelForVision2Seq, AutoProcessor
import torch
import torch.nn as nn
from peft import PeftModel, PeftConfig
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '../external/openvla')))
from prismatic.vla.datasets import RLDSBatchTransform
from prismatic.models.backbones.llm.prompting import PurePromptBuilder, VicunaV15ChatPromptBuilder
from prismatic.vla.action_tokenizer import ActionTokenizer

DEVICE_ID = 6 # GPU device id for rendering

## Load Processor & VLA
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
# base_model = AutoModelForVision2Seq.from_pretrained(
#     "openvla/openvla-7b",
#     attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,
#     low_cpu_mem_usage=True,
#     trust_remote_code=True
# ).to("cuda:6")

checkpoint_path = '/data2/zhaoyu/LIBERO_finetune/checkpoints/libero_spatial/openvla-b2-nproc3-imageAugFalse-0807'
vla = AutoModelForVision2Seq.from_pretrained(checkpoint_path, trust_remote_code=True, torch_dtype=torch.bfloat16).to(f"cuda:{DEVICE_ID}")

# Create Action Tokenizer
action_tokenizer = ActionTokenizer(processor.tokenizer)

Section 2: Use OpenVLA for inference

In [ ]:
%env MUJOCO_GL=osmesa
# os.environ['MUJOCO_GL'] = 'osmesa'
# must be set before importing mujoco_py for headless rendering
import os
import sys
import numpy as np
import random
import matplotlib.pyplot as plt
import torch
from PIL import Image
from IPython.display import display, clear_output

# Add VLA_DIR to PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '../')))
# Add LIBERO to PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '../external/LIBERO')))
from libero.libero.utils.time_utils import Timer
from libero.libero.utils.video_utils import VideoWriter
from libero.libero import benchmark, get_libero_path
from libero.libero.envs import OffScreenRenderEnv, SubprocVectorEnv
# from libero.lifelong.metric import (
#     evaluate_loss,
#     evaluate_success,
#     raw_obs_to_tensor_obs,
# )
# from libero.lifelong.utils import (
#     control_seed,
#     safe_device,
#     torch_load_model,
#     NpEncoder,
#     compute_flops,
# )
from utils.LIBERO_utils import get_task_names, extract_task_info, extract_env_obs

## User specific configurations
# TODO: change this into argparse for user input in python file
DATASET_NAME = "libero_spatial" # "libero_object", "libero_spatial", "libero_goal", "libero_10", "libero_90"
TASK_NAME = "pick_up_the_black_bowl_next_to_the_cookie_box_and_place_it_on_the_plate"
SAVE_VIDEO = True # save video of the evaluation process
EVAL_MAX_STEP = 500 # maximum number of steps for evaluation
PARALLEL_ENVS = 1 # number of parallel environments for evaluation
TEST_ENV = False # test environment with specific actions
if TEST_ENV:
    ACTION_PATH = "../demo_record/actions/KITCHEN_SCENE3_turn_on_the_stove_and_put_the_moka_pot_on_it_demo"

## Check evaluation configureations path
BDDL_FILES_BASE_PATH = get_libero_path("bddl_files")
INIT_STATES_BASE_PATH = get_libero_path("init_states")
VIDEO_FOLDER = "../eval_record/videos"
print("=====================================")
print("LIBERO evaluation BDDL files path: ", BDDL_FILES_BASE_PATH)
print("LIBERO evaluation initial states path: ", INIT_STATES_BASE_PATH)
if SAVE_VIDEO:
    print("Video saving to: ", VIDEO_FOLDER)
print("=====================================")


## Load evaluation environment
benchmark_dict = benchmark.get_benchmark_dict()
benchmark_instance = benchmark_dict[DATASET_NAME]()
tasks_num_eval = benchmark_instance.get_num_tasks() # number of tasks for evaluation
task_names_eval = benchmark_instance.get_task_names() # task names for evaluation
print(f"{tasks_num_eval} tasks for evaluation:\n {task_names_eval}")
# find the position for TASK_NAME in task_names_eval
task_id_eval = task_names_eval.index(TASK_NAME)

# load a task for evaluation
task_eval = benchmark_instance.get_task(task_id_eval)
language_instruction_eval = benchmark_instance.get_task(task_id_eval).language # language instruction for the task
print(language_instruction_eval) 

## Evaluation
with Timer() as t, VideoWriter(VIDEO_FOLDER, SAVE_VIDEO) as video_writer:
    env_args = {
        "bddl_file_name": os.path.join(
            BDDL_FILES_BASE_PATH, task_eval.problem_folder, task_eval.bddl_file
        ),
        "render_gpu_device_id": DEVICE_ID
    }

    env = SubprocVectorEnv(
        [lambda: OffScreenRenderEnv(**env_args) for _ in range(PARALLEL_ENVS)]
    )
        
    env.reset()
    env.seed(0)

    init_states_path = os.path.join(
        INIT_STATES_BASE_PATH, task_eval.problem_folder, task_eval.init_states_file
    )
    init_states = torch.load(init_states_path)
    indices = np.arange(PARALLEL_ENVS) % init_states.shape[0]
    init_states_ = init_states[indices]

    dones = [False] * PARALLEL_ENVS
    steps = 0
    obs = env.set_init_state(init_states_)
    
    for _ in range(10):  # simulate the physics without any actions
            env.step(np.zeros((PARALLEL_ENVS, 7)))
    
    print("Initial RGB observations:")
    fig, ax = plt.subplots()
    stacked_image = np.hstack([obs[k]["agentview_image"][::-1] for k in range(PARALLEL_ENVS)])
    im = ax.imshow(stacked_image)
    plt.axis('off')
    plt.show(block=False)

    ## Testing environment with specific actions
    '''
    taking 6min for 500 steps
    '''
    if TEST_ENV:
        actions = np.load(ACTION_PATH + "/0.npy")
        for timestep in range(actions.shape[0]):
            print(actions[timestep])
            obs, reward, done, info = env.step([actions[timestep]])
            video_writer.append_vector_obs(obs, done, camera_name="agentview_image")
        env.close()
    
    ## Use OpenVLA for evaluation
    else:
        for _ in range(EVAL_MAX_STEP):
            # collect image
            image = obs[0]["agentview_image"][::-1]
            image = Image.fromarray(image)
            prompt = f"In: What action should the robot take to {language_instruction_eval}?\nOut:"
            # Process the image and prompt
            with torch.no_grad():
                inputs = processor(prompt, image).to(f"cuda:{DEVICE_ID}", dtype=torch.bfloat16)
                # generate actions using OpenVLA
                action = vla.predict_action(**inputs, unnorm_key="libero_spatial", do_sample=False)
            action = np.array([action])
            # during OpenVLA training, gripper value was converted: -1 -> 1 for open; 1 -> 0 for close
            # convert back to original value
            # action[0][6] = 1 - 2 * action[0][6]
            # step the environment
            obs, reward, done, info = env.step(action)
            for k in range(PARALLEL_ENVS):
                dones[k] = dones[k] or done[k]
            if all(dones):
                break
            # record video
            video_writer.append_vector_obs(obs, done, camera_name="agentview_image")
            print(f"Step: {_} / {EVAL_MAX_STEP}, Action: {action}")
            # visualize every step as image
            # vis = np.hstack([obs[k]["agentview_image"][::-1] for k in range(PARALLEL_ENVS)])
            # im.set_data(vis)
            # clear_output(wait=True)  # Clear the previous output
            # display(fig)  # Display the updated figure
            # plt.pause(0.001)

            
        env.close()
    
    # TODO       
    # ## Use OpenVLA for evaluation with parallel environments
    # for _ in range(EVAL_MAX_STEP):
    #     # collect images from all parallel environments
    #     images_batch = [obs[k]["agentview_image"][::-1] for k in range(PARALLEL_ENVS)]
    #     images_batch = [
    #         Image.fromarray(image) if isinstance(image, np.ndarray) else image
    #         for image in images_batch
    #     ]
    #     images_batch = [image.convert("RGB") for image in images_batch]
    #     # generate prompts based on the language instruction
    #     prompts = [f"In: What action should the robot take to {language_instruction_eval}?\nOut:" for _ in range(PARALLEL_ENVS)]
    #     # Process the images and prompts
    #     inputs = processor(text=prompts, images=images_batch, return_tensors="pt", padding=True).to("cuda:2")
    #     for key, value in inputs.items():
    #         print(f"Shape of {key}: {value.shape}")
    #     # generate actions using OpenVLA
    #     with torch.no_grad():
    #         actions = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

    #     # step the environment
    #     obs, reward, done, info = env.step(actions)
        
    #     # record video
    #     video_writer.append_vector_obs(obs, done, camera_name="agentview_image")
